In [ ]:
import requests
import json
import pandas as pd
import datetime

response = requests.get("https://climatedataapi.azurewebsites.net/api/data")
df = pd.DataFrame(json.loads(response.content.decode("utf-8")))

df["datetime"] = pd.to_datetime(df.ti)
df["te_F"] = df.te * 9/5 + 32

print("Room Climate Data")
display(df)


timezone = datetime.timedelta(0,0,0,0,0,-7)
intial_date = datetime.datetime(2022, 2, 6, 23) - timezone

response = requests.get(f"https://api.weather.gov/stations/KIWA/observations?start={intial_date.isoformat()}Z")
raw_weather_data = json.loads(response.content.decode("utf-8"))

weather_data = []
for raw_data in raw_weather_data["features"]:
    temperature = raw_data["properties"]["temperature"]["value"]
    humidity = raw_data["properties"]["relativeHumidity"]["value"]
    timestamp = raw_data["properties"]["timestamp"]
    if (temperature is None) or (humidity is None) or (timestamp is None):
        continue
    weather_data.append({
        "timestamp": pd.to_datetime(timestamp) + timezone,
        "temperature": temperature * 9/5 + 32,
        "humidity": humidity
    })

weather_df = pd.DataFrame(weather_data).sort_values(by="timestamp")
print("\nWeather Climate Data")
display(weather_df)

In [ ]:
from plotly import graph_objects as go

figs = {}

def create_fig(fig_name, units):
    figs[fig_name] = go.Figure()
    figs[fig_name].update_layout(title=fig_name, xaxis_title="Time", yaxis_title=f"{fig_name} {units}", height=600)

create_fig("Temperature", "\N{DEGREE SIGN}F")
create_fig("Humidity", "(%)")

def create_trace(room, x, y):
    room_name = (room if room == "Weather" else f"{room} Bedroom" if room == "Master" else f"{room}'s Room")
    return go.Scatter(x=x, y=y, mode="lines+markers", text=room_name, name=room_name, line_shape="spline")

for room, room_data in df.groupby(by="rm"):
    figs["Temperature"].add_trace(create_trace(room, room_data.datetime, room_data.te_F))
    figs["Humidity"].add_trace(create_trace(room, room_data.datetime, room_data.hu))
figs["Temperature"].add_trace(create_trace("Weather", weather_df.timestamp, weather_df.temperature))
figs["Humidity"].add_trace(create_trace("Weather", weather_df.timestamp, weather_df.humidity))

for fig in figs:
    figs[fig].show()